# load_channels_static_to_bigquery

### 🎯Objetivo

Este notebook no transforma datos.
Este notebook tiene una única responsabilidad:
Cargar el DataFrame df_channels_snapshot (generado en el notebook 03) hacia BigQuery como tabla stática 

Destino:
youtube-datasets-360.angelgarciadatablog.channels_static

In [ ]:
from dotenv import load_dotenv
import os
from google.cloud import bigquery

load_dotenv()

PROJECT_ID = os.getenv("GCP_PROJECT")
DATASET_ID = "angelgarciadatablog"
TABLE_ID = "channels_static"

FULL_TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

client = bigquery.Client(project=PROJECT_ID)

print("Destino configurado:", FULL_TABLE_ID)

## 🧱 Cargar snapshot desde Parquet (temporal)

⚠️ Nota temporal:
Durante la fase de notebooks, el DataFrame se carga desde Parquet como mecanismo de intercambio entre notebooks.
En la versión productiva (scripts .py), el DataFrame se pasará directamente sin almacenamiento intermedio.

In [ ]:
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path.cwd().parents[0]
PROCESSED_PATH = PROJECT_ROOT / "data" / "processed" / "youtube"

file_path = PROCESSED_PATH / "channels_static.parquet"

df_channels_static = pd.read_parquet(file_path)

df_channels_static.head()


In [ ]:
df_channels_static.dtypes

## 🏗 Crear tabla particionada con el esquema y datos del dataframe 

In [ ]:
#Crear la tabla solo la primera vez, luego ya no se creará

from google.api_core.exceptions import NotFound
from google.cloud.bigquery import SchemaField

schema = [
    SchemaField("channel_id", "STRING"),
    SchemaField("channel_title", "STRING"),
    SchemaField("description", "STRING"),
    SchemaField("country", "STRING"),
    SchemaField("published_at", "TIMESTAMP"),
    SchemaField("thumbnail_url", "STRING"),
    SchemaField("channel_url", "STRING"),
    SchemaField("extracted_at", "TIMESTAMP"),
]

try:
    client.get_table(FULL_TABLE_ID)
    print("Tabla ya existe.")
    
except NotFound:
    table = bigquery.Table(FULL_TABLE_ID, schema=schema)
    client.create_table(table)
    print("Tabla creada.")


## 📌 Cargar datos del parquet a big query

In [ ]:
# 2️⃣ Carga los datos desde tu DataFrame hacia BigQuery. WRITE TRUNCATE = reescribe los datos
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE"
)

job = client.load_table_from_dataframe(
    df_channels_static,
    FULL_TABLE_ID,
    job_config=job_config
)

job.result()

print("Tabla static reemplazada correctamente.")